### Produce Constructor Standings

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_file_date","2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
race_results_df = spark.read.parquet(f'{presentation_folder_path}/race_results')\
    .filter(f"file_date = '{v_file_date}'")

In [0]:
race_year_list = df_column_to_list(race_results_df, 'race_year')

In [0]:
from pyspark.sql.functions import col

race_results_df = spark.read.parquet(f'{presentation_folder_path}/race_results')\
    .filter(col('race_year').isin(race_year_list))

In [0]:
display(race_results_df)

In [0]:
from pyspark.sql.functions import count, when, col, rank, desc, sum

In [0]:
constructor_df = race_results_df\
    .groupBy('race_year','team')\
    .agg(
        sum('points').alias('total_points'), 
        count(when(col('position') == 1, True)).alias("wins")
        )

In [0]:
display(constructor_df.filter('race_year == 2020'))

In [0]:
from pyspark.sql.window import Window

In [0]:
constructor_rank_spec = Window.partitionBy('race_year').orderBy(desc('total_points'), desc('wins'))

In [0]:
final_df = constructor_df.withColumn('rank', rank().over(constructor_rank_spec))

In [0]:
display(final_df.filter('race_year == 2020'))

In [0]:
# final_df.write.mode('overwrite').format('parquet').saveAsTable('f1_presentation.constructor_standings')

In [0]:
overwrite_partition(final_df, 'f1_presentation', 'constructor_standings', 'race_year')

In [0]:
%sql
SELECT COUNT(DISTINCT(race_year)) FROM f1_presentation.constructor_standings